<a href="https://colab.research.google.com/github/nomura-st/gas_questionnaire/blob/main/%E5%85%AC%E9%96%8B%E6%98%A0%E7%94%BB%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#############################
### カスタム情報
#############################
# 読み書き対象スプレッドシートID
ssID='11HxD-a2Sq8DXPEEiVfZmNtEqkdOxJKZckPRkkT0GOJY'
# 読み書き対象スプレッドシート シート名
sheetName='上映中'
# データ表が始まる行
rowStart=2

# 検索対象サイト
baseURL = 'https://eiga.com'
# 対象の県
prefURL = f'{baseURL}/theater/10'
paths = {
    "ユナイテッド・シネマ 前橋": "/100101/3229/",
    "109シネマズ高崎": "/100201/3230/",
    "イオンシネマ高崎": "/100201/3231/",
}


In [14]:
import requests
import re

from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート

def select(url, selector):
  html = requests.get(url)
  # 取得したHTMLをBeautifulSoupを使ってパースします。
  soup = BeautifulSoup(html.content, "html.parser")
  # パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
  elem = soup.select(selector)

  return elem

In [15]:
import re

def getMoviesFromEiga(url, path):
  # パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
  movies = select(url + path, "main .content-container section[data-title]")
  obj = []
  print("*********************************************")
  print(f'GET INFO from {path}')

  for movie in filter(lambda m: len(m.select("h2 a")) > 0,movies):
    print(f'TRY {movie["data-title"]}')
    m = {
      "name": movie["data-title"],
      "schedules": list(map(lambda t: {
          "type": t.select(".movie-type")[0].get_text("/") if len(t.select(".movie-type")) > 0 else "",
          "date": list(map(lambda d: d["data-time"], t.select("td [data-time]"))),
      }, movie.select(".movie-schedule"))),
      "time": "0",
    }

    if len(movie.select(".movie-image img")) > 0:
      m["image"] = movie.select(".movie-image img")[0]["src"]

    for i in movie.select(".movie-image .data span"):
      timeResult = re.match("(\d+)[分]*", i.get_text().strip())
      if  timeResult:
        m["time"] = timeResult.group(1)
      
    if len(movie.select("h2 a")) > 0:
      m["link"] = baseURL + movie.select("h2 a")[0]["href"]

    obj.append(m)
  
  return obj


In [16]:
#import json

theaters = list(map(lambda name: {
    "name": name,
    "movies": getMoviesFromEiga(prefURL, paths[name]),
}, paths.keys()))

# print (json.dumps(theaters, ensure_ascii=False))

*********************************************
GET INFO from /100101/3229/
TRY 映画　イチケイのカラス
TRY そして僕は途方に暮れる
TRY JSB3 LIVE FILM RISING SOUND
TRY THE FIRST SLAM DUNK
TRY アバター　ウェイ・オブ・ウォーター
TRY Dr.コトー診療所
TRY すずめの戸締まり
TRY ラーゲリより愛を込めて
TRY 非常宣言
TRY かがみの孤城
TRY 名探偵コナン　灰原哀物語　黒鉄のミステリートレイン
TRY ブラックナイトパレード
TRY 嘘八百　なにわ夢の陣
TRY ONE PIECE FILM RED
TRY トップガン　マーヴェリック
TRY 仮面ライダーギーツ×リバイス MOVIE バトルロワイヤル
TRY かぐや様は告らせたい　ファーストキッスは終わらない
*********************************************
GET INFO from /100201/3230/
TRY 映画　イチケイのカラス
TRY そして僕は途方に暮れる
TRY THE FIRST SLAM DUNK
TRY アバター　ウェイ・オブ・ウォーター
TRY Dr.コトー診療所
TRY すずめの戸締まり
TRY ラーゲリより愛を込めて
TRY 非常宣言
TRY 月の満ち欠け
TRY ブラックナイトパレード
TRY ONE PIECE FILM RED
TRY RRR
TRY 劇場版　転生したらスライムだった件　紅蓮の絆編
TRY 中島みゆき　劇場版　ライヴ・ヒストリー2
TRY 仮面ライダーギーツ×リバイス MOVIE バトルロワイヤル
*********************************************
GET INFO from /100201/3231/
TRY 映画　イチケイのカラス
TRY そして僕は途方に暮れる
TRY JSB3 LIVE FILM RISING SOUND
TRY ひみつのなっちゃん。
TRY ゲキ×シネ「神州無頼街」
TRY 妖怪ウォッチ♪　ジバニャンvsコマさん　もんげー大決戦だニャン
TRY THE FIRST SLAM DUNK
TRY アバター

In [17]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [18]:
import datetime

ss = gc.open_by_key(ssID)
sheet = ss.worksheet(sheetName)

# 既存を削除
ss.values_clear(f'\'{sheetName}\'!B2:J501')

row = rowStart
# 日時１つを１行として書き込み
for theater in theaters:
  print("*********************************************")
  print(f'THEATER: {theater["name"]}')

  for movie in theater["movies"]:
    print(f'MOVIE: {movie["name"]}')

    for schedule in movie["schedules"]:
      for d in schedule["date"]:
        dt = datetime.datetime.fromtimestamp(int(d), datetime.timezone.utc)
        # 日本のタイムゾーン調整
        dt = dt  + datetime.timedelta(hours = 9)
        # 終了時刻は目安 ＋１０分
        dtEnd = dt + datetime.timedelta(minutes = int(movie["time"]) + 10)
        dateStr = dt.strftime('%Y/%m/%d')
        timeStartStr = dt.strftime('%H:%M:%S')
        timeStartStrAbout = dt.strftime('%H時台(%p)')
        timeEndStr = dtEnd.strftime('%H:%M:%S')

        # 書き込みデータ
        ds= sheet.range(f'B{row}:J{row}')
        ds[0].value = movie["name"]
        ds[1].value = schedule["type"]
        ds[2].value = dateStr
        ds[3].value = timeStartStrAbout
        ds[4].value = timeStartStr
        ds[5].value = timeEndStr
        ds[6].value = movie["time"]
        ds[7].value = theater["name"]
        ds[8].value = movie["link"]
        # 書き込み
        sheet.update_cells(ds)

        row = row + 1


*********************************************
THEATER: ユナイテッド・シネマ 前橋
MOVIE: 映画　イチケイのカラス
MOVIE: そして僕は途方に暮れる
MOVIE: JSB3 LIVE FILM RISING SOUND
MOVIE: THE FIRST SLAM DUNK
MOVIE: アバター　ウェイ・オブ・ウォーター
MOVIE: Dr.コトー診療所
MOVIE: すずめの戸締まり
MOVIE: ラーゲリより愛を込めて
MOVIE: 非常宣言
MOVIE: かがみの孤城
MOVIE: 名探偵コナン　灰原哀物語　黒鉄のミステリートレイン
MOVIE: ブラックナイトパレード
MOVIE: 嘘八百　なにわ夢の陣
MOVIE: ONE PIECE FILM RED
MOVIE: トップガン　マーヴェリック
MOVIE: 仮面ライダーギーツ×リバイス MOVIE バトルロワイヤル
MOVIE: かぐや様は告らせたい　ファーストキッスは終わらない
*********************************************
THEATER: 109シネマズ高崎
MOVIE: 映画　イチケイのカラス
MOVIE: そして僕は途方に暮れる
MOVIE: THE FIRST SLAM DUNK
MOVIE: アバター　ウェイ・オブ・ウォーター
MOVIE: Dr.コトー診療所
MOVIE: すずめの戸締まり
MOVIE: ラーゲリより愛を込めて
MOVIE: 非常宣言
MOVIE: 月の満ち欠け
MOVIE: ブラックナイトパレード
MOVIE: ONE PIECE FILM RED
MOVIE: RRR
MOVIE: 劇場版　転生したらスライムだった件　紅蓮の絆編
MOVIE: 中島みゆき　劇場版　ライヴ・ヒストリー2
MOVIE: 仮面ライダーギーツ×リバイス MOVIE バトルロワイヤル
*********************************************
THEATER: イオンシネマ高崎
MOVIE: 映画　イチケイのカラス
MOVIE: そして僕は途方に暮れる
MOVIE: JSB3 LIVE FILM RISING SOUND
MOVIE: ひみつのなっちゃん。
M